In [1]:
import pandas as pd
from pathlib import Path
import sklearn as skl
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from collections import Counter
import pickle

In [2]:
stock_data = Path('StockAIClean3.csv')
stock_df = pd.read_csv(stock_data)
stock_df

,No.,Ticker,Company,Sector,Mkt Cap Cat,For P/E Cat,Dividend,EPS growth this year (%) Cat,EPS growth next year (%) Cat,EPS growth past 5 years (%) Cat,...,Float Short (%) Cat,Profit Margin (%) Cat,Performance (Year) (%) Cat,Employees Cat,Analyst Rec Cat,Team Rec Cat,Risk,Volume Cat,Var % Cat,Var%
0,1,AAPL,Apple Inc.,Technology,Mg Cap,Med,Yes,Yes,Yes,Yes,...,Lo,Pos,Up,Hi,Buy,Buy,Lo,Hi,Neg,-1.2
1,2,ACN,Accenture plc,Technology,Lg Cap,Med,Yes,Yes,Yes,Yes,...,Lo,Pos,Up,Hi,Buy,Buy,Lo,Med,Pos,5.3
2,3,ADBE,Adobe Inc.,Technology,Lg Cap,Hi,No,Yes,Yes,Yes,...,Lo,Pos,Up,Med,Buy,Buy,Lo,Med,Neg,-0.3
3,4,ADSK,"Autodesk, Inc.",Technology,Lg Cap,Med,No,Yes,Yes,Yes,...,Lo,Pos,Down,Med,Buy,Buy,Lo,Lo,Pos,9.1
4,5,AI,"C3.ai, Inc.",Technology,Mid Cap,Hi,No,No,Yes,No,...,Hi,Neg,Up,Sm,Hold,Hold,Med,Hi,Neg,-36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,68,UBER,"Uber Technologies, Inc.",Technology,Lg Cap,Hi,No,Yes,Yes,No,...,Lo,Neg,Up,Med,Buy,Hold,Med,Hi,Pos,12.7
68,69,UPST,"Upstart Holdings, Inc.",Financial,Mid Cap,Hi,No,No,Yes,No,...,Hi,Neg,Up,Sm,Hold,Buy,Lo,Hi,Neg,-68.2
69,70,VERI,"Veritone, Inc.",Technology,Mic Cap,Low,No,Yes,Yes,Yes,...,Hi,Neg,Down,Sm,Hold,Sell,Hi,Lo,Pos,5.8
70,71,VRSK,"Verisk Analytics, Inc.",Industrials,Lg Cap,Hi,Yes,Yes,Yes,Yes,...,Med,Pos,Up,Med,Buy,Hold,Med,Med,Pos,1.6


In [3]:
stock_df1 = stock_df.drop(columns=['No.', 'Ticker', 'Sector'])
stock_df1.head()

,Company,Mkt Cap Cat,For P/E Cat,Dividend,EPS growth this year (%) Cat,EPS growth next year (%) Cat,EPS growth past 5 years (%) Cat,EPS growth next 5 years (%) Cat,Sales growth past 5 years (%) Cat,Sales Cat,Float Short (%) Cat,Profit Margin (%) Cat,Performance (Year) (%) Cat,Employees Cat,Analyst Rec Cat,Team Rec Cat,Risk,Volume Cat,Var % Cat,Var%
0,Apple Inc.,Mg Cap,Med,Yes,Yes,Yes,Yes,Yes,Yes,Hi,Lo,Pos,Up,Hi,Buy,Buy,Lo,Hi,Neg,-1.2
1,Accenture plc,Lg Cap,Med,Yes,Yes,Yes,Yes,Yes,Yes,Hi,Lo,Pos,Up,Hi,Buy,Buy,Lo,Med,Pos,5.3
2,Adobe Inc.,Lg Cap,Hi,No,Yes,Yes,Yes,Yes,Yes,Med,Lo,Pos,Up,Med,Buy,Buy,Lo,Med,Neg,-0.3
3,"Autodesk, Inc.",Lg Cap,Med,No,Yes,Yes,Yes,Yes,Yes,Med,Lo,Pos,Down,Med,Buy,Buy,Lo,Lo,Pos,9.1
4,"C3.ai, Inc.",Mid Cap,Hi,No,No,Yes,No,Yes,Yes,Lo,Hi,Neg,Up,Sm,Hold,Hold,Med,Hi,Neg,-36.0


In [4]:
stock_df2 = pd.get_dummies(stock_df1, columns=["Team Rec Cat", "Risk", "Analyst Rec Cat", "Mkt Cap Cat", "For P/E Cat", "Dividend", "EPS growth this year (%) Cat", "EPS growth next year (%) Cat", "EPS growth past 5 years (%) Cat", "EPS growth next 5 years (%) Cat", "Sales growth past 5 years (%) Cat", "Sales Cat", "Float Short (%) Cat", "Profit Margin (%) Cat", "Performance (Year) (%) Cat", "Employees Cat", "Volume Cat", "Var % Cat"])
stock_df2

,Company,Var%,Team Rec Cat_Buy,Team Rec Cat_Hold,Team Rec Cat_Sell,Risk_Hi,Risk_Lo,Risk_Med,Analyst Rec Cat_Buy,Analyst Rec Cat_Hold,...,Performance (Year) (%) Cat_Down,Performance (Year) (%) Cat_Up,Employees Cat_Hi,Employees Cat_Med,Employees Cat_Sm,Volume Cat_Hi,Volume Cat_Lo,Volume Cat_Med,Var % Cat_Neg,Var % Cat_Pos
0,Apple Inc.,-1.2,1,0,0,0,1,0,1,0,...,0,1,1,0,0,1,0,0,1,0
1,Accenture plc,5.3,1,0,0,0,1,0,1,0,...,0,1,1,0,0,0,0,1,0,1
2,Adobe Inc.,-0.3,1,0,0,0,1,0,1,0,...,0,1,0,1,0,0,0,1,1,0
3,"Autodesk, Inc.",9.1,1,0,0,0,1,0,1,0,...,1,0,0,1,0,0,1,0,0,1
4,"C3.ai, Inc.",-36.0,0,1,0,0,0,1,0,1,...,0,1,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"Uber Technologies, Inc.",12.7,0,1,0,0,0,1,1,0,...,0,1,0,1,0,1,0,0,0,1
68,"Upstart Holdings, Inc.",-68.2,1,0,0,0,1,0,0,1,...,0,1,0,0,1,1,0,0,1,0
69,"Veritone, Inc.",5.8,0,0,1,1,0,0,0,1,...,1,0,0,0,1,0,1,0,0,1
70,"Verisk Analytics, Inc.",1.6,0,1,0,0,0,1,1,0,...,0,1,0,1,0,0,0,1,0,1


In [5]:
X = stock_df2.drop(columns=['Company'])
y = stock_df2['Company']

In [6]:
X.columns

Index(['Var%', 'Team Rec Cat_Buy', 'Team Rec Cat_Hold', 'Team Rec Cat_Sell',
       'Risk_Hi', 'Risk_Lo', 'Risk_Med', 'Analyst Rec Cat_Buy',
       'Analyst Rec Cat_Hold', 'Mkt Cap Cat_Lg Cap', 'Mkt Cap Cat_Mg Cap',
       'Mkt Cap Cat_Mic Cap', 'Mkt Cap Cat_Mid Cap', 'Mkt Cap Cat_Sm Cap',
       'For P/E Cat_Hi', 'For P/E Cat_Low', 'For P/E Cat_Med', 'Dividend_No',
       'Dividend_Yes', 'EPS growth this year (%) Cat_No',
       'EPS growth this year (%) Cat_Yes', 'EPS growth next year (%) Cat_No',
       'EPS growth next year (%) Cat_Yes',
       'EPS growth past 5 years (%) Cat_No',
       'EPS growth past 5 years (%) Cat_Yes',
       'EPS growth next 5 years (%) Cat_No',
       'EPS growth next 5 years (%) Cat_Yes',
       'Sales growth past 5 years (%) Cat_No',
       'Sales growth past 5 years (%) Cat_Yes', 'Sales Cat_Hi', 'Sales Cat_Lo',
       'Sales Cat_Med', 'Float Short (%) Cat_Hi', 'Float Short (%) Cat_Lo',
       'Float Short (%) Cat_Med', 'Profit Margin (%) Cat_Neg',
    

In [7]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1, stratify=y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
# create a scaler instance
scaler = StandardScaler()

In [ ]:
# fit/train the scaler
X_scaler = scaler.fit(X_train)

In [ ]:
# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [ ]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)


In [ ]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

Training Data Score: 0.6666666666666666
Testing Data Score: 0.5555555555555556


c:\Users\soobi\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\soobi\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [ ]:
predictions = model.predict(X_test_scaled)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,Hold,Buy
1,Buy,Buy
2,Buy,Hold
3,Hold,Buy
4,Buy,Buy
5,Buy,Hold
6,Hold,Hold
7,Hold,Hold
8,Buy,Buy
9,Hold,Buy


In [ ]:
# Display the accuracy score for the test dataset.
score = balanced_accuracy_score(y_test, predictions)
print('Accuracy score: ', score)

Accuracy score:  0.6111111111111112


In [ ]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,6,3
Actual 1,4,5


In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         Buy       0.60      0.67      0.63         9
        Hold       0.62      0.56      0.59         9

    accuracy                           0.61        18
   macro avg       0.61      0.61      0.61        18
weighted avg       0.61      0.61      0.61        18



In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data
rf_predictions = rf_model.predict(X_test_scaled)
rf_results = pd.DataFrame({"Prediction": rf_predictions, "Actual": y_test}).reset_index(drop=True)
rf_results

,Prediction,Actual
0,Hold,Buy
1,Buy,Buy
2,Hold,Hold
3,Buy,Buy
4,Buy,Buy
5,Buy,Hold
6,Hold,Hold
7,Hold,Hold
8,Buy,Buy
9,Hold,Buy


In [ ]:
# Display the accuracy score for the test dataset.
rf_score = balanced_accuracy_score(y_test, rf_predictions)
print('Accuracy score: ', rf_score)

Accuracy score:  0.7222222222222222


In [ ]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, rf_predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,7,2
Actual 1,3,6


In [ ]:
print(classification_report(y_test, rf_predictions))

              precision    recall  f1-score   support

         Buy       0.70      0.78      0.74         9
        Hold       0.75      0.67      0.71         9

    accuracy                           0.72        18
   macro avg       0.72      0.72      0.72        18
weighted avg       0.73      0.72      0.72        18



In [ ]:
# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resample, y_resample = ros.fit_resample(X_train, y_train)

Counter(y_resample)
y_resample.value_counts()


Buy     27
Hold    27
Name: Team Rec Cat, dtype: int64

In [ ]:
# Creating the decision tree classifier instance
model1 = tree.DecisionTreeClassifier()
model1 = model1.fit(X_resample, y_resample)


In [ ]:
predictions1 = model1.predict(X_test_scaled)
results1 = pd.DataFrame({"Prediction": predictions1, "Actual": y_test}).reset_index(drop=True)
results1.head(10)

c:\Users\soobi\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


,Prediction,Actual
0,Hold,Buy
1,Buy,Buy
2,Buy,Hold
3,Buy,Buy
4,Buy,Buy
5,Buy,Hold
6,Hold,Hold
7,Hold,Hold
8,Buy,Buy
9,Hold,Buy


In [ ]:
# Display the accuracy score for the test dataset.
score1 = balanced_accuracy_score(y_test, predictions1)
print('Accuracy score: ', score1)

Accuracy score:  0.6666666666666667


In [ ]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions1)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,5,4
Actual 1,2,7


In [ ]:
print(classification_report(y_test, predictions1))

              precision    recall  f1-score   support

         Buy       0.71      0.56      0.63         9
        Hold       0.64      0.78      0.70         9

    accuracy                           0.67        18
   macro avg       0.68      0.67      0.66        18
weighted avg       0.68      0.67      0.66        18



In [ ]:
# Create a random forest classifier
rf_model1 = RandomForestClassifier(n_estimators=500, random_state=78)

In [ ]:
# Fitting the model
rf_model1 = rf_model1.fit(X_resample, y_resample)


In [ ]:
# Making predictions using the testing data
rf_predictions1 = rf_model1.predict(X_test_scaled)
rf_results1 = pd.DataFrame({"Prediction": rf_predictions1, "Actual": y_test}).reset_index(drop=True)
rf_results1

c:\Users\soobi\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Prediction,Actual
0,Hold,Buy
1,Buy,Buy
2,Hold,Hold
3,Buy,Buy
4,Buy,Buy
5,Buy,Hold
6,Hold,Hold
7,Hold,Hold
8,Buy,Buy
9,Hold,Buy


In [ ]:
# Display the accuracy score for the test dataset.
rf_score1 = balanced_accuracy_score(y_test, rf_predictions1)
print('Accuracy score: ', rf_score1)

Accuracy score:  0.7222222222222222


In [ ]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, rf_predictions1)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,6,3
Actual 1,2,7


In [ ]:
print(classification_report(y_test, rf_predictions1))

              precision    recall  f1-score   support

         Buy       0.75      0.67      0.71         9
        Hold       0.70      0.78      0.74         9

    accuracy                           0.72        18
   macro avg       0.72      0.72      0.72        18
weighted avg       0.73      0.72      0.72        18



In [ ]:
# import pickle

# with open("scaler.pkl", "wb") as f:
#     pickle.dump(X_scaler,f)

# with open("model.pkl", "wb") as f:
#     pickle.dump(model,f)

# with open("model1.pkl", "wb") as f:
#     pickle.dump(model1,f)